In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

def load_data(csv_path):
    """Carrega o dataset Parquet"""
    print("📊 Carregando dados...")
    df = pd.read_csv(csv_path)
    print(f"✅ Dataset carregado: {df.shape[0]} linhas × {df.shape[1]} colunas")
    return df

def basic_info(df):
    """Informações básicas do dataset"""
    print("\n" + "="*50)
    print("📋 INFORMAÇÕES BÁSICAS")
    print("="*50)

    print(f"• Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
    print(f"• Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    print("\n📊 Tipos de dados:")
    print(df.dtypes.value_counts())

    print("\n🔍 Primeiras 5 linhas:")
    print(df.head())

    print("\n⏩ Últimas 5 linhas:")
    print(df.tail())

def missing_analysis(df):
    """Análise de valores missing"""
    print("\n" + "="*50)
    print("🔍 ANÁLISE DE VALORES MISSING")
    print("="*50)

    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100

    missing_df = pd.DataFrame({
        'Coluna': missing.index,
        'Valores Missing': missing.values,
        'Percentual (%)': missing_percent.values
    }).sort_values('Valores Missing', ascending=False)

    print(missing_df[missing_df['Valores Missing'] > 0])

    if missing.sum() == 0:
        print("✅ Nenhum valor missing encontrado!")
    else:
        # Visualização
        plt.figure(figsize=(10, 6))
        missing_df[missing_df['Valores Missing'] > 0].plot(
            x='Coluna', y='Percentual (%)', kind='bar',
            title='Percentual de Valores Missing por Coluna'
        )
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

def categorical_analysis(df):
    """Análise de variáveis categóricas"""
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    print("\n" + "="*50)
    print("📈 ANÁLISE DE VARIÁVEIS CATEGÓRICAS")
    print("="*50)
    print(f"• Número de variáveis categóricas: {len(categorical_cols)}")

    for col in categorical_cols:
        print(f"\n🎯 Coluna: {col}")
        print(f"   • Valores únicos: {df[col].nunique()}")
        print(f"   • Top 5 valores mais frequentes:")
        print(df[col].value_counts().head())

        # Visualização para colunas com poucas categorias
        if df[col].nunique() <= 15:
            plt.figure(figsize=(10, 6))
            df[col].value_counts().plot(kind='bar', title=f'Distribuição de {col}')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

def numerical_analysis(df):
    """Análise de variáveis numéricas"""
    numerical_cols = df.select_dtypes(include=[np.number]).columns

    print("\n" + "="*50)
    print("🔢 ANÁLISE DE VARIÁVEIS NUMÉRICAS")
    print("="*50)
    print(f"• Número de variáveis numéricas: {len(numerical_cols)}")

    if len(numerical_cols) > 0:
        # Estatísticas descritivas
        print("\n📊 Estatísticas Descritivas:")
        print(df[numerical_cols].describe())

        # Histogramas
        n_cols = min(3, len(numerical_cols))
        n_rows = (len(numerical_cols) + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

        for i, col in enumerate(numerical_cols):
            if i < len(axes):
                df[col].hist(bins=30, ax=axes[i])
                axes[i].set_title(f'Distribuição de {col}')
                axes[i].set_xlabel(col)
                axes[i].set_ylabel('Frequência')

        # Esconder eixos vazios
        for i in range(len(numerical_cols), len(axes)):
            axes[i].set_visible(False)

        plt.tight_layout()
        plt.show()

        # Boxplots para identificar outliers
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

        for i, col in enumerate(numerical_cols):
            if i < len(axes):
                df.boxplot(column=col, ax=axes[i])
                axes[i].set_title(f'Boxplot de {col}')

        for i in range(len(numerical_cols), len(axes)):
            axes[i].set_visible(False)

        plt.tight_layout()
        plt.show()

def correlation_analysis(df):
    """Análise de correlação"""
    numerical_cols = df.select_dtypes(include=[np.number]).columns

    if len(numerical_cols) > 1:
        print("\n" + "="*50)
        print("📈 ANÁLISE DE CORRELAÇÃO")
        print("="*50)

        # Matriz de correlação
        corr_matrix = df[numerical_cols].corr()

        print("\n📊 Matriz de Correlação:")
        print(corr_matrix)

        # Heatmap
        plt.figure(figsize=(10, 8))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm',
                   center=0, square=True, linewidths=0.5)
        plt.title('Matriz de Correlação - Variáveis Numéricas')
        plt.tight_layout()
        plt.show()

        # Correlações fortes (abs > 0.7)
        strong_corr = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                if abs(corr_matrix.iloc[i, j]) > 0.7:
                    strong_corr.append((
                        corr_matrix.columns[i],
                        corr_matrix.columns[j],
                        corr_matrix.iloc[i, j]
                    ))

        if strong_corr:
            print("\n🔥 Correlações Fortes (|r| > 0.7):")
            for col1, col2, corr in strong_corr:
                print(f"   • {col1} ↔ {col2}: {corr:.3f}")

def outlier_analysis(df):
    """Identificação de outliers"""
    numerical_cols = df.select_dtypes(include=[np.number]).columns

    print("\n" + "="*50)
    print("🚨 ANÁLISE DE OUTLIERS")
    print("="*50)

    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        n_outliers = len(outliers)

        if n_outliers > 0:
            print(f"• {col}: {n_outliers} outliers ({n_outliers/len(df)*100:.2f}%)")
            print(f"  Limites: [{lower_bound:.2f}, {upper_bound:.2f}]")

def data_quality_report(df):
    """Relatório de qualidade dos dados"""
    print("\n" + "="*50)
    print("📋 RELATÓRIO DE QUALIDADE DOS DADOS")
    print("="*50)

    report = []
    for col in df.columns:
        col_report = {
            'Coluna': col,
            'Tipo': df[col].dtype,
            'Valores Únicos': df[col].nunique(),
            'Valores Missing': df[col].isnull().sum(),
            '% Missing': (df[col].isnull().sum() / len(df)) * 100,
            'Zeros': (df[col] == 0).sum() if df[col].dtype in [np.number] else 0
        }

        if df[col].dtype in [np.number]:
            col_report.update({
                'Média': df[col].mean(),
                'Mediana': df[col].median(),
                'Desvio Padrão': df[col].std()
            })
        else:
            col_report.update({
                'Moda': df[col].mode().iloc[0] if not df[col].empty else None,
                'Valor Mais Frequente': df[col].value_counts().index[0] if not df[col].empty else None
            })

        report.append(col_report)

    report_df = pd.DataFrame(report)
    print(report_df.to_string(index=False))

def main():
    """Função principal"""
    # Caminho para seu arquivo Parquet
    csv_path = "./dataset_completo.csv"  # Altere para o caminho correto

    try:
        # Carregar dados
        df = load_data(csv_path)

        # Executar análises
        # Verificar valores infinitos
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        inf_count = (df[numeric_cols] == np.inf).sum().sum() + (df[numeric_cols] == -np.inf).sum().sum()
        print(f"Valores infinitos: {inf_count}")

        print("\n" + "="*50)
        print("✅ ANÁLISE EXPLORATÓRIA CONCLUÍDA!")
        print("="*50)

    except FileNotFoundError:
        print(f"❌ Arquivo {csv_path} não encontrado!")
        print("Verifique o caminho do arquivo.")
    except Exception as e:
        print(f"❌ Erro durante a análise: {e}")

if __name__ == "__main__":
    main()

📊 Carregando dados...
✅ Dataset carregado: 220406 linhas × 84 colunas
Valores infinitos: 3074

✅ ANÁLISE EXPLORATÓRIA CONCLUÍDA!


### Tratamento dos Dados


In [4]:
csv_path = "./dataset_completo.csv"  # Ajuste se necessário
try:
    df = load_data(csv_path)
except:
  print('dataset não encontrado')

df

📊 Carregando dados...
✅ Dataset carregado: 220406 linhas × 84 colunas


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.137.250-35.175.71.102-51438-443-6,192.168.137.250,51438,35.175.71.102,443,6,08/10/2022 06:45:21 AM,31334,3,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,192.168.137.250-54.167.177.211-44438-443-6,192.168.137.250,44438,54.167.177.211,443,6,08/10/2022 06:45:21 AM,109939,2,2,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,54.167.177.211-192.168.137.250-443-44438-6,54.167.177.211,443,192.168.137.250,44438,6,08/10/2022 06:45:21 AM,1,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,192.168.137.250-35.175.71.102-51438-443-6,192.168.137.250,51438,35.175.71.102,443,6,08/10/2022 06:45:21 AM,114090,1,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,192.168.137.253-173.198.192.103-41834-4431-6,192.168.137.253,41834,173.198.192.103,4431,6,08/10/2022 06:45:25 AM,181,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220401,192.168.137.66-192.168.137.131-50884-80-6,192.168.137.66,50884,192.168.137.131,80,6,08/08/2022 01:26:36 PM,0,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
220402,192.168.137.131-192.168.137.66-80-51216-6,192.168.137.131,80,192.168.137.66,51216,6,08/08/2022 01:26:36 PM,25145,7,1,...,44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
220403,192.168.137.66-192.168.137.131-51216-80-6,192.168.137.66,51216,192.168.137.131,80,6,08/08/2022 01:26:36 PM,0,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
220404,192.168.137.66-192.168.137.131-50812-80-6,192.168.137.66,50812,192.168.137.131,80,6,08/08/2022 01:26:02 PM,34779202,13,6,...,20,19593852.0,0.0,19593852.0,19593852.0,11027529.0,0.0,11027529.0,11027529.0,1


In [5]:
df.describe()

,Src Port,Dst Port,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,220406.000000,220406.000000,220406.000000,2.204060e+05,220406.000000,220406.000000,2.204060e+05,2.204060e+05,220406.000000,220406.000000,...,220406.000000,2.204060e+05,2.204060e+05,2.204060e+05,2.204060e+05,2.204060e+05,2.204060e+05,2.204060e+05,2.204060e+05,220406.000000
mean,35423.957116,13851.499247,11.422407,2.418231e+07,86.723950,9.984764,9.331489e+03,2.462000e+03,255.291376,50.237385,...,17.029555,8.324395e+05,2.349374e+05,1.130953e+06,6.564968e+05,8.260562e+06,8.024696e+05,9.074767e+06,7.591142e+06,0.585860
std,21488.210440,20096.244979,5.720626,4.114345e+07,2111.749835,252.319153,2.088086e+05,3.333661e+05,982.200166,133.931171,...,11.084817,3.485316e+06,1.398825e+06,4.389401e+06,3.172419e+06,1.820295e+07,4.394253e+06,1.945140e+07,1.778140e+07,0.492574
min,0.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,15909.000000,68.000000,6.000000,5.668725e+04,1.000000,1.000000,4.000000e+00,0.000000e+00,4.000000,0.000000,...,8.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,42999.000000,443.000000,17.000000,3.012125e+05,2.000000,1.000000,6.200000e+01,4.000000e+01,48.000000,4.000000,...,8.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
75%,52229.000000,32100.000000,17.000000,3.016963e+07,5.000000,4.000000,3.180000e+02,1.540000e+02,175.000000,48.000000,...,32.000000,7.202750e+03,0.000000e+00,8.295000e+03,2.159000e+03,5.904990e+06,0.000000e+00,6.048929e+06,5.264532e+06,1.000000
max,65534.000000,65535.000000,17.000000,1.200000e+08,187446.000000,80558.000000,4.041864e+07,1.164379e+08,36200.000000,2896.000000,...,60.000000,1.127263e+08,6.734190e+07,1.127263e+08,1.127263e+08,1.199993e+08,7.315624e+07,1.199993e+08,1.199993e+08,1.000000


In [18]:
for i in df.columns:
  if df[i].isna().sum() > 0:
    print(df[i].isna().sum() )



2898


np.int64(0)

In [61]:
import pandas as pd
import numpy as np

# Carregar o dataset (ajuste o caminho se necessário)
df = pd.read_csv("dataset_completo.csv")

# Selecionar apenas colunas numéricas
num_cols = df.select_dtypes(include=[np.number]).columns

# Criar uma máscara booleana para detectar valores infinitos
mask_inf = np.isinf(df[num_cols])

# Filtrar linhas que possuem pelo menos um valor infinito
df_inf = df[mask_inf.any(axis=1)]

# ---- Parte 1: visualizar dados infinitos ----
print(f"Número de linhas com infinito: {df_inf.shape[0]}")
print(f"Colunas com infinito: {list(mask_inf.any()[mask_inf.any()].index)}")

# Salvar os dados com infinito (opcional)
df_inf.to_csv("dados_infinito.csv", index=False)

# Exibir as primeiras linhas com infinito
print("\nPrimeiras linhas com infinito:")
print(df_inf.head())


df_inf.columns

Número de linhas com infinito: 2986
Colunas com infinito: ['Flow Bytes/s', 'Flow Packets/s']

Primeiras linhas com infinito:
                                           Flow ID           Src IP  Src Port  \
408      35.175.71.102-192.168.137.148-443-54072-6    35.175.71.102       443   
3149  192.168.137.253-173.198.192.103-42496-4431-6  192.168.137.253     42496   
3225  192.168.137.253-173.198.192.103-42540-4431-6  192.168.137.253     42540   
4037  192.168.137.253-173.198.192.103-42674-4431-6  192.168.137.253     42674   
4122  192.168.137.253-173.198.192.103-42722-4431-6  192.168.137.253     42722   

               Dst IP  Dst Port  Protocol               Timestamp  \
408   192.168.137.148     54072         6  08/10/2022 06:51:31 AM   
3149  173.198.192.103      4431         6  08/10/2022 07:22:31 AM   
3225  173.198.192.103      4431         6  08/10/2022 07:23:31 AM   
4037  173.198.192.103      4431         6  08/10/2022 07:32:32 AM   
4122  173.198.192.103      4431         6  

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Len

In [62]:
#Taxa de valores inf
df_inf[[ 'Flow Bytes/s', 'Flow Packets/s', 'Label']][df_inf.Label==1].count()/df_inf[[ 'Flow Packets/s', 'Label']].count()

,0
Flow Bytes/s,NaN
Flow Packets/s,0.98493
Label,0.98493


In [63]:
# ---- Parte 2: tratar os infinitos substituindo pelo maior valor finito da coluna ----
for col in num_cols:
    if np.isinf(df[col]).any():
        max_val = df.loc[~np.isinf(df[col]), col].max()
        df[col] = df[col].replace([np.inf, -np.inf], max_val)
        print(f" Coluna '{col}' corrigida: ∞ substituído por {max_val}")

# Agora df está limpo
print("\n Dataset final corrigido!")


 Coluna 'Flow Bytes/s' corrigida: ∞ substituído por 2896000000.0
 Coluna 'Flow Packets/s' corrigida: ∞ substituído por 3000000.0

 Dataset final corrigido!


In [56]:
# Colunas com valores ausentes
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.isna().sum()[df.isna().sum() > 0]


,0
Flow Bytes/s,2898


In [66]:

# Taxa de valores ausentes por label com valor 1
print('Taxa de valores ausentes:',df['Label'][df['Flow Bytes/s'].isna()][df['Label']==1].count()/df['Label'][df['Flow Bytes/s'].isna()].count())
#tratamento
print('substituição:',df['Flow Bytes/s'].max())
df['Flow Bytes/s'] = df['Flow Bytes/s'].replace([np.nan], df['Flow Bytes/s'].max())
print('Valores ausentes finais:',df['Label'][df['Flow Bytes/s'].isna()])

Taxa de valores ausentes: nan
substituição: 2896000000.0
Valores ausentes finais: Series([], Name: Label, dtype: int64)


2896000000.0

,Label


In [70]:
df.to_csv('dataset_tratado_completo.csv', index=False)

In [71]:
df = pd.read_csv('dataset_tratado_completo.csv')
df

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.137.250-35.175.71.102-51438-443-6,192.168.137.250,51438,35.175.71.102,443,6,08/10/2022 06:45:21 AM,31334,3,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,192.168.137.250-54.167.177.211-44438-443-6,192.168.137.250,44438,54.167.177.211,443,6,08/10/2022 06:45:21 AM,109939,2,2,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,54.167.177.211-192.168.137.250-443-44438-6,54.167.177.211,443,192.168.137.250,44438,6,08/10/2022 06:45:21 AM,1,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,192.168.137.250-35.175.71.102-51438-443-6,192.168.137.250,51438,35.175.71.102,443,6,08/10/2022 06:45:21 AM,114090,1,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,192.168.137.253-173.198.192.103-41834-4431-6,192.168.137.253,41834,173.198.192.103,4431,6,08/10/2022 06:45:25 AM,181,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220401,192.168.137.66-192.168.137.131-50884-80-6,192.168.137.66,50884,192.168.137.131,80,6,08/08/2022 01:26:36 PM,0,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
220402,192.168.137.131-192.168.137.66-80-51216-6,192.168.137.131,80,192.168.137.66,51216,6,08/08/2022 01:26:36 PM,25145,7,1,...,44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
220403,192.168.137.66-192.168.137.131-51216-80-6,192.168.137.66,51216,192.168.137.131,80,6,08/08/2022 01:26:36 PM,0,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
220404,192.168.137.66-192.168.137.131-50812-80-6,192.168.137.66,50812,192.168.137.131,80,6,08/08/2022 01:26:02 PM,34779202,13,6,...,20,19593852.0,0.0,19593852.0,19593852.0,11027529.0,0.0,11027529.0,11027529.0,1


In [73]:
print('Valores ausentes finais:',df['Label'][df['Flow Bytes/s'].isna()])
# Criar uma máscara booleana para detectar valores infinitos
mask_inf = np.isinf(df[num_cols])

# Filtrar linhas que possuem pelo menos um valor infinito
df_inf= df[mask_inf.any(axis=1)]
df_inf

Valores ausentes finais: Series([], Name: Label, dtype: int64)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
